In [1]:
import xarray as xr

In [2]:
# open forcing file
ds = xr.open_dataset('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929.nc')
ds

<xarray.Dataset>
Dimensions:      (lat: 360, lon: 288, time: 167, bounds: 2)
Coordinates:
  * lat          (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon          (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time         (time) object 0000-12-31 12:00:00 ... 0166-01-01 12:00:00
Dimensions without coordinates: bounds
Data variables:
    time_bnds    (time, bounds) object ...
    date         (time) float64 ...
    CO2_LBC      (time, lat) float32 ...
    N2O_LBC      (time, lat) float32 ...
    CH4_LBC      (time, lat) float32 ...
    CFC11eq_LBC  (time, lat) float32 ...
    CF2CL2_LBC   (time, lat) float32 ...
Attributes:
    data_title:     \nLower boundary conditions for CMIP6 1pctCO2 simulations...
    data_summary:   \nTime-dependent mole fraction surface boundary condition...
    data_creator:   \nMike Mills, mmills@ucar.edu
    cesm_contact:   \nMike Mills, mmills@ucar.edu
    creation_date:  \nSat Sep 29 18:46:46 MDT 2018
    data_script:    \ncreated with program CMIP6_1pctCO2GlobAnnAvg.ncl\nfrom ...

In [3]:
# grab relevant arrays to use for updated co2 forcing

# co2 coordinates
lat = ds['lat'].values
time = ds['time'].values

# reverse order of co2
CO2_LBC = ds['CO2_LBC'].values[::-1]

In [4]:
# new co2 dataarray
new_co2 = xr.DataArray(CO2_LBC, coords={'time': time, 'lat': lat,}, dims=['time', 'lat'])

In [6]:
new_co2 # sanity check

<xarray.DataArray (time: 167, lat: 360)>
array([[0.00145381, 0.00145381, 0.00145381, ..., 0.00145381, 0.00145381,
        0.00145381],
       [0.00145381, 0.00145381, 0.00145381, ..., 0.00145381, 0.00145381,
        0.00145381],
       [0.00143942, 0.00143942, 0.00143942, ..., 0.00143942, 0.00143942,
        0.00143942],
       ...,
       [0.00028716, 0.00028716, 0.00028716, ..., 0.00028716, 0.00028716,
        0.00028716],
       [0.00028432, 0.00028432, 0.00028432, ..., 0.00028432, 0.00028432,
        0.00028432],
       [0.00028432, 0.00028432, 0.00028432, ..., 0.00028432, 0.00028432,
        0.00028432]], dtype=float32)
Coordinates:
  * time     (time) object 0000-12-31 12:00:00 ... 0166-01-01 12:00:00
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75

In [7]:
# update the co2 variable in the original dataset
ds['CO2_LBC'] = new_co2

In [8]:
# add update attributes
ds.attrs['updated_by'] = 'Maria J. Molina (molina@ucar.edu)'
ds.attrs['update_scope'] = 'reversed order of CO2_LBC using python xarray'
ds.attrs['updated_date'] = 'Wed Dec 08 MDT 2021'

In [9]:
ds # sanity check

<xarray.Dataset>
Dimensions:      (lat: 360, lon: 288, time: 167, bounds: 2)
Coordinates:
  * lat          (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon          (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time         (time) object 0000-12-31 12:00:00 ... 0166-01-01 12:00:00
Dimensions without coordinates: bounds
Data variables:
    time_bnds    (time, bounds) object 0000-12-31 00:00:00 ... 0166-01-02 00:...
    date         (time) float64 1.231e+03 1.07e+04 ... 1.651e+06 1.66e+06
    CO2_LBC      (time, lat) float32 0.001454 0.001454 ... 0.0002843 0.0002843
    N2O_LBC      (time, lat) float32 2.73e-07 2.73e-07 ... 2.73e-07 2.73e-07
    CH4_LBC      (time, lat) float32 8.082e-07 8.082e-07 ... 8.082e-07 8.082e-07
    CFC11eq_LBC  (time, lat) float32 3.211e-11 3.211e-11 ... 3.211e-11 3.211e-11
    CF2CL2_LBC   (time, lat) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    data_title:     \nLower boundary conditions for CMIP6 1pctCO2 simulations...
    data_summary:   \nTime-dependent mole fraction surface boundary condition...
    data_creator:   \nMike Mills, mmills@ucar.edu
    cesm_contact:   \nMike Mills, mmills@ucar.edu
    creation_date:  \nSat Sep 29 18:46:46 MDT 2018
    data_script:    \ncreated with program CMIP6_1pctCO2GlobAnnAvg.ncl\nfrom ...
    updated_by:     Maria J. Molina (molina@ucar.edu)
    update_scope:   reversed order of CO2_LBC using python xarray
    updated_date:   Wed Dec 08 MDT 2021

In [10]:
# save the updated file
ds.to_netcdf('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929_REV.nc')